# DeepRoof Checkpoint Inference on One PNG

This notebook runs model inference on one image (`/workspace/test.png`) using a checkpoint and shows segmentation results visually.

In [ ]:
from __future__ import annotations

import json
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch

In [ ]:
PROJECT_ROOT = Path('/workspace/roof')
CONFIG_PATH = PROJECT_ROOT / 'configs' / 'deeproof_scratch_swin_L.py'
CHECKPOINT_PATH = PROJECT_ROOT / 'work_dirs' / 'swin_l_scratch_v1' / 'iter_8000.pth'
INPUT_IMAGE_PATH = Path('/workspace/test.png')

OUTPUT_DIR = PROJECT_ROOT / 'outputs' / 'checkpoint_inference'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
OVERLAY_PATH = OUTPUT_DIR / 'test_segmentation_overlay.png'
SEM_MASK_PATH = OUTPUT_DIR / 'test_semantic_mask.png'
SUMMARY_PATH = OUTPUT_DIR / 'test_inference_summary.json'

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

print(f'CONFIG: {CONFIG_PATH}')
print(f'CHECKPOINT: {CHECKPOINT_PATH}')
print(f'INPUT: {INPUT_IMAGE_PATH}')
print(f'DEVICE: {DEVICE}')

In [ ]:
for p in (CONFIG_PATH, CHECKPOINT_PATH, INPUT_IMAGE_PATH):
    if not p.exists():
        raise FileNotFoundError(f'Path not found: {p}')

img_bgr = cv2.imread(str(INPUT_IMAGE_PATH), cv2.IMREAD_COLOR)
if img_bgr is None:
    raise RuntimeError(f'Could not load image: {INPUT_IMAGE_PATH}')

img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
H, W = img_rgb.shape[:2]
print(f'Loaded image: {W}x{H}')

plt.figure(figsize=(8, 8))
plt.imshow(img_rgb)
plt.title('Input image')
plt.axis('off')
plt.show()

In [ ]:
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from mmseg.utils import register_all_modules
from mmseg.apis import init_model, inference_model

register_all_modules(init_default_scope=False)

# Ensure custom modules are imported and registered.
import deeproof.models.backbones.swin_v2_compat
import deeproof.models.deeproof_model
import deeproof.models.heads.mask2former_head
import deeproof.models.heads.geometry_head
import deeproof.models.losses

model = init_model(str(CONFIG_PATH), str(CHECKPOINT_PATH), device=DEVICE)
model.eval()
print('Model loaded successfully.')

In [ ]:
result = inference_model(model, str(INPUT_IMAGE_PATH))
if isinstance(result, (list, tuple)):
    result = result[0]

# Semantic map
if hasattr(result, 'pred_sem_seg') and hasattr(result.pred_sem_seg, 'data'):
    sem_map = result.pred_sem_seg.data.squeeze(0).detach().cpu().numpy().astype(np.uint8)
else:
    sem_map = np.zeros((H, W), dtype=np.uint8)

if sem_map.shape != (H, W):
    sem_map = cv2.resize(sem_map, (W, H), interpolation=cv2.INTER_NEAREST)

# Instances (if available)
masks = np.zeros((0, H, W), dtype=bool)
scores = np.array([], dtype=np.float32)
labels = np.array([], dtype=np.int64)
if hasattr(result, 'pred_instances') and result.pred_instances is not None:
    inst = result.pred_instances
    if hasattr(inst, 'masks') and inst.masks is not None:
        masks_t = inst.masks
        if torch.is_tensor(masks_t):
            masks = masks_t.detach().cpu().numpy().astype(bool)
    if hasattr(inst, 'scores') and inst.scores is not None:
        scores = inst.scores.detach().cpu().numpy()
    if hasattr(inst, 'labels') and inst.labels is not None:
        labels = inst.labels.detach().cpu().numpy()

print(f'Unique semantic classes: {np.unique(sem_map).tolist()}')
print(f'Predicted instances: {len(masks)}')
if len(scores) > 0:
    print(f'Score range: {float(scores.min()):.4f} .. {float(scores.max()):.4f}')

In [ ]:
# Palette: background, flat_roof, sloped_roof
palette = np.array([
    [0, 0, 0],
    [0, 255, 0],
    [255, 0, 0],
], dtype=np.uint8)

sem_vis = palette[np.clip(sem_map, 0, len(palette) - 1)]
overlay = cv2.addWeighted(img_rgb, 0.60, sem_vis, 0.40, 0.0)

# Draw instance contours and scores if present
for i, mask in enumerate(masks):
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(overlay, contours, -1, (255, 255, 255), 1)

    if len(contours) > 0 and i < len(scores):
        c = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(c)
        cv2.putText(
            overlay,
            f'{float(scores[i]):.2f}',
            (x, max(0, y - 5)),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.4,
            (255, 255, 255),
            1,
            cv2.LINE_AA,
        )

cv2.imwrite(str(OVERLAY_PATH), cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))
cv2.imwrite(str(SEM_MASK_PATH), sem_map)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.imshow(img_rgb)
plt.title('Input')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(overlay)
plt.title('Segmentation Overlay')
plt.axis('off')
plt.show()

print(f'Saved overlay: {OVERLAY_PATH}')
print(f'Saved semantic mask: {SEM_MASK_PATH}')

In [ ]:
summary = {
    'input_image': str(INPUT_IMAGE_PATH),
    'config': str(CONFIG_PATH),
    'checkpoint': str(CHECKPOINT_PATH),
    'image_size': [int(H), int(W)],
    'semantic_classes': [int(x) for x in np.unique(sem_map)],
    'instance_count': int(len(masks)),
    'score_mean': float(scores.mean()) if len(scores) > 0 else None,
    'score_max': float(scores.max()) if len(scores) > 0 else None,
    'overlay_path': str(OVERLAY_PATH),
    'semantic_mask_path': str(SEM_MASK_PATH),
}

with SUMMARY_PATH.open('w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2)

print(json.dumps(summary, indent=2))
print(f'Saved summary: {SUMMARY_PATH}')